In [1]:
import pandas as pd
import os
from zipfile import ZipFile

In [59]:
def unzipping(directory):
    for zipFile in os.listdir(directory):
        filePath=directory+"//"+zipFile
        zf = ZipFile(filePath, 'r')
        zf.extractall(directory)
        zf.close()

In [60]:
unzipping("May")

In [63]:
def sampling_merge(directory):
    dfs=pd.DataFrame()
    for filename in os.listdir(directory):
        path=directory+"//"+filename
        df=pd.read_csv(path, header=None)
        df_sample=df.sample(frac=0.05)
        dfs=dfs.append(df_sample)
    dfs.to_csv(directory+"//"+directory+"_sample.csv", index=False, header=None)
    print("Done.")

In [64]:
sampling_merge("May")

Done.


In [1]:
# def sampling(directory):
#     for filename in os.listdir(directory):
#         path=directory+"//"+filename
#         df=pd.read_csv(path, header=None)
#         df_sample=df.sample(frac=0.10)
#         df_sample.to_csv(directory+"//sample_"+filename, index=False, header=None)
#     print("Done.")

In [15]:
# sample from the total month
def month_subsetting(monthDirectory, monthHydratedFile):
    path = monthDirectory+"//"+monthHydratedFile
    df_month = pd.read_csv(path)
    chunk_size = int(df_month.shape[0]/4)
    i = 1
    for start in range(0, df_month.shape[0], chunk_size):
        df_subset= df_month.iloc[start: start + chunk_size]
        #df_subset= df_subset[(df_subset['retweet_count']>0) | (df_subset['favorite_count']>0)]
        df_subset.to_csv(monthDirectory+"//_subset_"+str(i)+".csv", index=False)
        i += 1

In [ ]:
month_subsetting("May","May_sample.csv")
print("Subsetting Done.")

In [7]:
df_month.columns

Index(['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'retweet_count', 'retweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_screen_name', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name.1',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified'],
      dtype='object')